In [ ]:
from groq import Groq
import os
from typing import Dict

os.environ["GROQ_API_KEY"] = "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"

client = Groq(api_key=os.getenv("GROQ_API_KEY"))


In [ ]:
import json

json_file_path = 'datasets/patient_records1.json'

with open(json_file_path, 'r', encoding='utf-8') as file:
    test_data = json.load(file)

In [27]:
def groq_pipeline_exists_nl(patient_record: str, semantic_regex_task: str, model: str = "qwen-2.5-32b") -> bool:
    prompt = f"""
    Patient Data (Natural Language):
    {patient_record}

    Context:
    Extract semantic symbols from the provided patient data and determine if a match exists.

    Task:
    Determine whether at least one match of the following semantic regex is implied or explicitly present in the patient data above.

    Semantic Regex:
    {semantic_regex_task}

    Return exactly one word: "True" or "False".
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You strictly answer True or False based on patient data and provided semantic regex matching."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0,
        max_tokens=5,
        top_p=1,
        stream=False,
        stop=None,
    )

    result = response.choices[0].message.content.strip().lower()

    return result == "true"


In [ ]:
from tqdm import tqdm

predictions = []
labels = []

for item in tqdm(test_data, desc="Evaluating data"):
    patient_record = item["record"]
    semantic_regex = item["s_regex"]
    true_label = item["match"]
    
    pred = groq_pipeline_exists_nl(
        patient_record=patient_record,
        semantic_regex_task=semantic_regex
    )
    
    predictions.append(pred)
    labels.append(item["match"])


Evaluating data: 100%|██████████| 20/20 [00:07<00:00,  2.57it/s]


In [33]:
accuracy = sum([p == l for p, l in zip(predictions, labels)]) / len(labels)
print(f"Accuracy: {accuracy:.2%}")


Accuracy: 100.00%
